In [1]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD #most common namespaces
import urllib.parse #for parsing strings to URI's
import numpy as np

In [2]:
df = pd.read_csv('record_linkage_with_pos.csv',sep=',',quotechar='"')
print(df)
#df['savant_team'].value_counts()

     savant_id  schedule_id        savant_uri  \
0            0       547989        Abreu_Jose   
1            1       660670   Acuna_Jr_Ronald   
2            2       642715      Adames_Willy   
3            3       542583     Aguilar_Jesus   
4            4       645261   Alcantara_Sandy   
..         ...          ...               ...   
169        169       621550    Wisdom_Patrick   
170        170       677951     Witt_Jr_Bobby   
171        171       657140       Wright_Kyle   
172        172       573262  Yastrzemski_Mike   
173        173       592885  Yelich_Christian   

                          schedule_url       savant_name schedule_name  \
0    https://www.mlb.com/player/547989        Jose Abreu       Abreu J   
1    https://www.mlb.com/player/660670   Ronald Acuna Jr     Acuna Jr.   
2    https://www.mlb.com/player/642715      Willy Adames        Adames   
3    https://www.mlb.com/player/542583     Jesus Aguilar       Aguilar   
4    https://www.mlb.com/player/645261   

In [3]:
print(df['savant_team'].unique())

['Chicago White Sox' 'Atlanta Braves' 'Milwaukee Brewers' 'Miami Marlins'
 'New York Mets' 'Houston Astros' 'Los Angeles Dodgers'
 'Oakland Athletics' 'St. Louis Cardinals' 'Tampa Bay Rays'
 'Minnesota Twins' 'Detroit Tigers' 'Washington Nationals'
 'Kansas City Royals' 'Toronto Blue Jays' 'Cleveland Guardians'
 'Colorado Rockies' 'Boston Red Sox' 'Philadelphia Phillies'
 'New York Yankees' 'Seattle Mariners' 'San Diego Padres'
 'San Francisco Giants' 'Cincinnati Reds' 'Arizona Diamondbacks'
 'Texas Rangers' 'Chicago Cubs' 'Pittsburgh Pirates' 'Baltimore Orioles'
 'Los Angeles Angels']


In [4]:
df2 = pd.read_csv('team_schedule_result.csv',sep=',',quotechar='"')
print(df2)
#df2['Team'].value_counts()

        Id                   Team TimeInfo                Against  \
0        0         Atlanta Braves    04/07        Cincinnati Reds   
1        1        Cincinnati Reds    04/07         Atlanta Braves   
2        2           Chicago Cubs    04/07      Milwaukee Brewers   
3        3      Milwaukee Brewers    04/07           Chicago Cubs   
4        4       New York Yankees    04/08         Boston Red Sox   
...    ...                    ...      ...                    ...   
1211  1211         Houston Astros    11/02  Philadelphia Phillies   
1212  1212  Philadelphia Phillies    11/03         Houston Astros   
1213  1213         Houston Astros    11/03  Philadelphia Phillies   
1214  1214         Houston Astros    11/05  Philadelphia Phillies   
1215  1215  Philadelphia Phillies    11/05         Houston Astros   

     SimpleResult SimpleLocation  ScoreSecured  ScoreGiven  \
0            LOSE           HOME             3           6   
1             WIN           AWAY             6 

In [5]:
df3 = pd.read_csv('player_overall_performance.csv',sep=',',quotechar='"')
print(df3)

         Id PlayerType   W   L  ERA   G  GS  SV  IP  SO  ...     AB     R  \
0    592669     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  474.0  62.0   
1    669257     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  508.0  68.0   
2    672386     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  470.0  59.0   
3    571448     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  557.0  73.0   
4    664761     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  586.0  79.0   
..      ...        ...  ..  ..  ...  ..  ..  ..  ..  ..  ...    ...   ...   
168  542583     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  464.0  39.0   
169  680757     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  563.0  89.0   
170  668804     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  542.0  74.0   
171  677594     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  511.0  84.0   
172  642715     Batter NaN NaN  NaN NaN NaN NaN NaN NaN  ...  563.0  83.0   

         H    HR    RBI    SB    AVG    OBP    SLG    OPS  
0    121.0  29.

In [6]:
split_performance = ['monthly','game_type','platoon','inning']
df4 = []

for i in range(0, 4):
    df4_temp = pd.read_csv('player_overall_performance_split_'+str(split_performance[i])+'.csv',sep=',',quotechar='"')
    df4_temp = df4_temp.replace(np.nan, 0.0).replace('-.--', 0.0)
    #print(df4_temp)
    df4.append(df4_temp)

In [7]:
df5 = pd.read_csv('player_in_match.csv',sep=',',quotechar='"')
print(df5)

        Id  PlayerId PlayerType  MatchId   IP    H    R   ER   AB  R.1  H.1  \
0        0    621566     Batter        0  NaN  NaN  NaN  NaN  2.0  1.0  0.0   
1        1    663586     Batter        0  NaN  NaN  NaN  NaN  4.0  1.0  3.0   
2        2    542303     Batter        0  NaN  NaN  NaN  NaN  3.0  0.0  0.0   
3        3    621020     Batter        0  NaN  NaN  NaN  NaN  4.0  0.0  0.0   
4        4    608331    Pitcher        0  5.2  8.0  5.0  5.0  NaN  NaN  NaN   
...    ...       ...        ...      ...  ...  ...  ...  ...  ...  ...  ...   
5491  5491    656941     Batter     1215  NaN  NaN  NaN  NaN  3.0  1.0  1.0   
5492  5492    656555     Batter     1215  NaN  NaN  NaN  NaN  4.0  0.0  0.0   
5493  5493    592663     Batter     1215  NaN  NaN  NaN  NaN  3.0  0.0  1.0   
5494  5494    592206     Batter     1215  NaN  NaN  NaN  NaN  4.0  0.0  0.0   
5495  5495    664761     Batter     1215  NaN  NaN  NaN  NaN  3.0  0.0  1.0   

      RBI  
0     0.0  
1     2.0  
2     0.0  
3  

In [8]:
df6 = pd.read_csv('inference_result_match.csv',sep=',',quotechar='"')
print(df6)

      MatchId InferenceResult
0        1215             WIN
1        1214             WIN
2        1213             WIN
3        1212             WIN
4        1211             WIN
...       ...             ...
1011     1201             WIN
1012     1200             WIN
1013     1209             WIN
1014     1208             WIN
1015     1207             WIN

[1016 rows x 2 columns]


In [9]:
df7 = pd.read_csv('inference_result_player.csv',sep=',',quotechar='"')
print(df7)

      PlayerId PlayerType  MatchId  InferenceResult
0       592669     Batter      709              0.0
1       592669     Batter     1065              0.0
2       592669     Batter     1096              0.0
3       592669     Batter      362              0.0
4       592669     Batter      681              0.0
...        ...        ...      ...              ...
4588    642715     Batter     1078              0.0
4589    642715     Batter      426              1.0
4590    642715     Batter      613              0.0
4591    642715     Batter      669              0.0
4592    642715     Batter      807              1.0

[4593 rows x 4 columns]


In [10]:
g = Graph()
ppl = Namespace('http://dsci558.org/player/')
loc = Namespace('http://mylocations.org/addresses/')
schema = Namespace('http://schema.org/')
ttm = Namespace('http://dsci558.org/team/')
mmc = Namespace('http://dsci558.org/match/')
ccs = Namespace('http://dsci558.org/class/')
cct = Namespace('http://dsci558.org/cluster/')
ont = Namespace('http://dsci558.org/ontology/')
rdf0 = Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')

g.bind('bkgpy', ppl)
g.bind('bkgtm', ttm)
g.bind('bkgmc', mmc)
g.bind('bkgcs', ccs)
g.bind('bkgct', cct)
g.bind('bkgot', ont)
g.bind('rdf', rdf0)
g.bind('rdfs', rdfs)
g.bind('foaf', FOAF)
g.bind('xsd', XSD)

In [11]:
# team general info
for row_team in df['savant_team'].unique():
    this_row_team = row_team.replace(' ','_')
    g.add((URIRef(ttm+this_row_team), RDF.type, FOAF.Group ))
    g.add((URIRef(ttm+this_row_team), URIRef(schema+'name'), Literal(row_team, datatype=XSD.string) ))
    g.add((URIRef(ttm+this_row_team), URIRef(ont+'has_match'), URIRef(cct+this_row_team+'_match') ))
    g.add((URIRef(cct+this_row_team+"_match"), RDF.type, URIRef(ccs+'Cluster') ))
    g.add((URIRef(ttm+this_row_team), URIRef(ont+'has_player'), URIRef(cct+this_row_team+'_player') ))
    g.add((URIRef(cct+this_row_team+"_player"), RDF.type, URIRef(ccs+'Cluster') ))

# player general info
for index, row in df.iterrows():
    this_row_id = str(row['schedule_id'])
    g.add((URIRef(ppl+this_row_id), RDF.type, FOAF.Person))
    g.add((URIRef(ppl+this_row_id), URIRef(schema+'name'), Literal(row['official_name'], datatype=XSD.string) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'uri'), Literal(row['savant_uri'], datatype=XSD.anyURI) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'url'), Literal(row['schedule_url'], datatype=XSD.anyURI) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'belong_to_team'), URIRef(ttm+str(row['savant_team']).replace(' ','_')) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'position'), Literal(row['Position'], datatype=XSD.string) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'pos_bat'), Literal(row['Pos_Bat'], datatype=XSD.string) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'pos_throw'), Literal(row['Pos_Throw'], datatype=XSD.string) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'has_status'), URIRef(cct+this_row_id+'_split') ))
    g.add((URIRef(cct+this_row_id+'_split'), RDF.type, URIRef(ccs+'Cluster') ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'has_match'), URIRef(cct+this_row_id+'_match') ))
    g.add((URIRef(cct+this_row_id+'_match'), RDF.type, URIRef(ccs+'Cluster') ))
    g.add((URIRef(cct+str(row['savant_team']).replace(' ','_')+'_player'), URIRef(ont+'for_player'), (URIRef(ppl+this_row_id)) ))

# match general info
for index, row in df2.iterrows():
    this_row_team = str(row['Team']).replace(' ','_')
    #g.add((URIRef(ttm+this_row_team), URIRef(ont+'has_match'), URIRef(mmc+str(row['GeneralInfo']).replace(' ','_')) ))
    g.add((URIRef(cct+this_row_team+'_match'), URIRef(ont+'for_match'), URIRef(mmc+str(row['Id'])) ))
    g.add((URIRef(mmc+str(row['Id'])), RDF.type, URIRef(ccs+'Match') ))
    g.add((URIRef(mmc+str(row['Id'])), URIRef(ont+'plays_by'), URIRef(ttm+str(row['Team']).replace(' ','_')) ))
    g.add((URIRef(mmc+str(row['Id'])), URIRef(ont+'plays_on'), Literal('2022-'+str(row['TimeInfo']).replace('/','-'), datatype=XSD.date) ))
    g.add((URIRef(mmc+str(row['Id'])), URIRef(ont+'plays_against'), URIRef(ttm+str(row['Against']).replace(' ','_')) ))
    g.add((URIRef(mmc+str(row['Id'])), URIRef(ont+'ends_with'), Literal(row['SimpleResult'], datatype=XSD.string) ))
    g.add((URIRef(mmc+str(row['Id'])), URIRef(ont+'plays_at'), Literal(row['SimpleLocation'], datatype=XSD.string) ))
    g.add((URIRef(mmc+str(row['Id'])), URIRef(ont+'has_score_secured'), Literal(row['ScoreSecured'], datatype=XSD.integer) ))
    g.add((URIRef(mmc+str(row['Id'])), URIRef(ont+'has_score_given'), Literal(row['ScoreGiven'], datatype=XSD.integer) ))

# player overall performance
for index, row in df3.iterrows():
    this_row_id = str(row['Id'])
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'has_type'), URIRef(cct+row['PlayerType']) ))
    g.add((URIRef(ppl+this_row_id), URIRef(ont+'has_status'), URIRef(cct+this_row_id+'_overall') ))
    g.add((URIRef(cct+this_row_id+'_overall'), RDF.type, URIRef(ccs+'Cluster') ))
    if 'Pitcher' in row['PlayerType']:
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_W'), Literal(row['W'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_L'), Literal(row['L'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_ERA'), Literal(row['ERA'], datatype=XSD.float) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_G'), Literal(row['G'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_GS'), Literal(row['GS'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_SV'), Literal(row['SV'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_IP'), Literal(row['IP'], datatype=XSD.float) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_SO'), Literal(row['SO'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_WHIP'), Literal(row['WHIP'], datatype=XSD.float) ))
    else:
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_PA'), Literal(row['PA'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_AB'), Literal(row['AB'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_R'), Literal(row['R'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_H'), Literal(row['H'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_HR'), Literal(row['HR'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_RBI'), Literal(row['RBI'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_SB'), Literal(row['SB'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_AVG'), Literal(row['AVG'], datatype=XSD.float) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_OBP'), Literal(row['OBP'], datatype=XSD.float) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_SLG'), Literal(row['SLG'], datatype=XSD.float) ))
        g.add((URIRef(cct+this_row_id+'_overall'), URIRef(ont+'has_OPS'), Literal(row['OPS'], datatype=XSD.float) ))

# player overall performance _split
for i in range(0, 4):
    for index, row in df4[i].iterrows():
        this_row_id = str(row['Id'])
        g.add((URIRef(cct+this_row_id+'_split'), URIRef(ont+'has_status'), URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')) ))
        g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), RDF.type, URIRef(ccs+'Cluster') ))
        if 'Pitcher' in row['PlayerType']:
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_W'), Literal(row['W'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_L'), Literal(row['L'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_ERA'), Literal(row['ERA'], datatype=XSD.float) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_G'), Literal(row['G'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_GS'), Literal(row['GS'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_SV'), Literal(row['SV'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_IP'), Literal(row['IP'], datatype=XSD.float) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_BF'), Literal(row['BF'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_H'), Literal(row['H'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_R'), Literal(row['R'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_ER'), Literal(row['ER'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_BB'), Literal(row['BB'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_SO'), Literal(row['SO'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_WHIP'), Literal(row['WHIP'], datatype=XSD.float) ))
        else:
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_PA'), Literal(row['PA'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_AB'), Literal(row['AB'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_R'), Literal(row['R.1'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_H'), Literal(row['H.1'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_2B'), Literal(row['2B'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_3B'), Literal(row['3B'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_HR'), Literal(row['HR'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_RBI'), Literal(row['RBI'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_BB'), Literal(row['BB.1'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_SO'), Literal(row['SO.1'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_SB'), Literal(row['SB'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_CS'), Literal(row['CS'], datatype=XSD.integer) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_HBP'), Literal(row['HBP'], datatype=XSD.float) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_AVG'), Literal(row['AVG'], datatype=XSD.float) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_OBP'), Literal(row['OBP'], datatype=XSD.float) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_SLG'), Literal(row['SLG'], datatype=XSD.float) ))
            g.add((URIRef(cct+this_row_id+'_'+str(row['MonthType']).replace(' ','_')), URIRef(ont+'has_OPS'), Literal(row['OPS'], datatype=XSD.float) ))


# player performance in match
for index, row in df5.iterrows():
    this_row_id = str(row['PlayerId'])
    g.add((URIRef(cct+this_row_id+'_match'), URIRef(ont+'for_match'), URIRef(mmc+str(row['MatchId'])) ))
    g.add((URIRef(mmc+str(row['MatchId'])), URIRef(ont+'has_player_played'), URIRef(ppl+this_row_id) ))

    g.add((URIRef(cct+this_row_id+'_match'), URIRef(ont+'has_status'), URIRef(cct+this_row_id+'_'+str(row['MatchId'])) ))
    g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), RDF.type, URIRef(ccs+'Cluster') ))

    g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_status_for_match'), URIRef(mmc+str(row['MatchId'])) ))

    if 'Pitcher' in row['PlayerType']:
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_IP'), Literal(row['IP'], datatype=XSD.float) ))
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_H'), Literal(row['H'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_R'), Literal(row['R'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_ER'), Literal(row['ER'], datatype=XSD.integer) ))
    else:
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_AB'), Literal(row['AB'], datatype=XSD.float) ))
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_R'), Literal(row['R.1'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_H'), Literal(row['H.1'], datatype=XSD.integer) ))
        g.add((URIRef(cct+this_row_id+'_'+str(row['MatchId'])), URIRef(ont+'has_RBI'), Literal(row['RBI'], datatype=XSD.integer) ))


# inference result
for index, row in df6.iterrows():
    g.add((URIRef(mmc+str(row['MatchId'])), URIRef(ont+'inference-ends_with'), Literal(row['InferenceResult'], datatype=XSD.string) ))
for index, row in df7.iterrows():
    if 'Pitcher' in row['PlayerType']:
        g.add((URIRef(cct+str(row['PlayerId'])+'_'+str(row['MatchId'])), URIRef(ont+'inference-has_ER'), Literal(row['InferenceResult'], datatype=XSD.integer) ))
    else:
        g.add((URIRef(cct+str(row['PlayerId'])+'_'+str(row['MatchId'])), URIRef(ont+'inference-has_RBI'), Literal(row['InferenceResult'], datatype=XSD.integer) ))


#print(g.serialize(format='turtle'))
g.serialize('prototype.ttl',format='turtle')

<Graph identifier=Nc50938b4786f46e1bf8c6fc85ec30f65 (<class 'rdflib.graph.Graph'>)>

In [12]:
g = Graph()
ppl = Namespace('http://dsci558.org/player/')
loc = Namespace('http://mylocations.org/addresses/')
schema = Namespace('http://schema.org/')
ttm = Namespace('http://dsci558.org/team/')
mmc = Namespace('http://dsci558.org/match/')
ccs = Namespace('http://dsci558.org/class/')
cct = Namespace('http://dsci558.org/cluster/')
ont = Namespace('http://dsci558.org/ontology/')
rdf0 = Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')

g.bind('bkgpy', ppl)
g.bind('bkgtm', ttm)
g.bind('bkgmc', mmc)
g.bind('bkgcs', ccs)
g.bind('bkgct', cct)
g.bind('bkgot', ont)
g.bind('rdf', rdf0)
g.bind('rdfs', rdfs)
g.bind('foaf', FOAF)
g.bind('xsd', XSD)

g.add((URIRef(ccs+'Cluster'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(ccs+'Match'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(ttm+'team'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(ppl+'player'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(mmc+'match'), RDF.type, URIRef(rdfs+'Class') ))

g.add((URIRef(ont+'has_match'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_match'), URIRef(rdfs+'label'), Literal("Has Match", datatype=XSD.string) ))
g.add((URIRef(ont+'has_match'), URIRef(rdfs+'domain'), URIRef(ttm+'team') ))
g.add((URIRef(ont+'has_match'), URIRef(rdfs+'range'), URIRef(cct+'team'+'_match') ))

g.add((URIRef(ont+'has_player'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_player'), URIRef(rdfs+'label'), Literal("Has Player", datatype=XSD.string) ))
g.add((URIRef(ont+'has_player'), URIRef(rdfs+'domain'), URIRef(ttm+'team') ))
g.add((URIRef(ont+'has_player'), URIRef(rdfs+'range'), URIRef(cct+'team'+'_player') ))

g.add((URIRef(schema+'name'), RDF.type, RDF.Property ))
g.add((URIRef(schema+'name'), URIRef(rdfs+'label'), Literal("Name", datatype=XSD.string) ))
g.add((URIRef(schema+'name'), URIRef(rdfs+'domain'), URIRef(ttm+'team') ))
g.add((URIRef(schema+'name'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(schema+'name'), RDF.type, RDF.Property ))
g.add((URIRef(schema+'name'), URIRef(rdfs+'label'), Literal("Name", datatype=XSD.string) ))
g.add((URIRef(schema+'name'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(schema+'name'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(ont+'uri'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'uri'), URIRef(rdfs+'label'), Literal("URI", datatype=XSD.string) ))
g.add((URIRef(ont+'uri'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'uri'), URIRef(rdfs+'range'), XSD.anyURI ))

g.add((URIRef(ont+'url'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'url'), URIRef(rdfs+'label'), Literal("URL", datatype=XSD.string) ))
g.add((URIRef(ont+'url'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'url'), URIRef(rdfs+'range'), XSD.anyURI ))

g.add((URIRef(ont+'belong_to_team'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'belong_to_team'), URIRef(rdfs+'label'), Literal("Belong to Team", datatype=XSD.string) ))
g.add((URIRef(ont+'belong_to_team'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'belong_to_team'), URIRef(rdfs+'range'), URIRef(ttm+'team') ))

g.add((URIRef(ont+'position'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'position'), URIRef(rdfs+'label'), Literal("Plays Position", datatype=XSD.string) ))
g.add((URIRef(ont+'position'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'position'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(ont+'pos_bat'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'pos_bat'), URIRef(rdfs+'label'), Literal("Bat with", datatype=XSD.string) ))
g.add((URIRef(ont+'pos_bat'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'pos_bat'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(ont+'pos_throw'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'pos_throw'), URIRef(rdfs+'label'), Literal("Throw with", datatype=XSD.string) ))
g.add((URIRef(ont+'pos_throw'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'pos_throw'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(ont+'has_status'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_status'), URIRef(rdfs+'label'), Literal("Has Status", datatype=XSD.string) ))
g.add((URIRef(ont+'has_status'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'has_status'), URIRef(rdfs+'range'), URIRef(cct+'player'+'_split') ))

g.add((URIRef(ont+'has_match'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_match'), URIRef(rdfs+'label'), Literal("Has Match", datatype=XSD.string) ))
g.add((URIRef(ont+'has_match'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'has_match'), URIRef(rdfs+'range'), URIRef(cct+'player'+'_match') ))

g.add((URIRef(ont+'for_player'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'for_player'), URIRef(rdfs+'label'), Literal("For Player", datatype=XSD.string) ))
g.add((URIRef(ont+'for_player'), URIRef(rdfs+'domain'), URIRef(cct+'team'+'_player') ))
g.add((URIRef(ont+'for_player'), URIRef(rdfs+'range'), URIRef(ppl+'player') ))


g.add((URIRef(ont+'for_match'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'for_match'), URIRef(rdfs+'label'), Literal("For Match", datatype=XSD.string) ))
g.add((URIRef(ont+'for_match'), URIRef(rdfs+'domain'), URIRef(cct+'team'+'_match') ))
g.add((URIRef(ont+'for_match'), URIRef(rdfs+'range'), URIRef(mmc+'match') ))

g.add((URIRef(ont+'plays_by'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'plays_by'), URIRef(rdfs+'label'), Literal("Plays by", datatype=XSD.string) ))
g.add((URIRef(ont+'plays_by'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'plays_by'), URIRef(rdfs+'range'), URIRef(ttm+'team') ))

g.add((URIRef(ont+'plays_on'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'plays_on'), URIRef(rdfs+'label'), Literal("Plays on", datatype=XSD.string) ))
g.add((URIRef(ont+'plays_on'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'plays_on'), URIRef(rdfs+'range'), XSD.date ))

g.add((URIRef(ont+'plays_against'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'plays_against'), URIRef(rdfs+'label'), Literal("Plays against", datatype=XSD.string) ))
g.add((URIRef(ont+'plays_against'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'plays_against'), URIRef(rdfs+'range'), URIRef(ttm+'team') ))

g.add((URIRef(ont+'ends_with'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'ends_with'), URIRef(rdfs+'label'), Literal("Ends with", datatype=XSD.string) ))
g.add((URIRef(ont+'ends_with'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'ends_with'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(ont+'inference-ends_with'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'inference-ends_with'), URIRef(rdfs+'label'), Literal("Inference- Ends with", datatype=XSD.string) ))
g.add((URIRef(ont+'inference-ends_with'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'inference-ends_with'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(ont+'plays_at'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'plays_at'), URIRef(rdfs+'label'), Literal("Plays at", datatype=XSD.string) ))
g.add((URIRef(ont+'plays_at'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'plays_at'), URIRef(rdfs+'range'), XSD.string ))

g.add((URIRef(ont+'has_score_secured'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_score_secured'), URIRef(rdfs+'label'), Literal("Has Score Secured", datatype=XSD.string) ))
g.add((URIRef(ont+'has_score_secured'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'has_score_secured'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_score_given'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_score_given'), URIRef(rdfs+'label'), Literal("Has Score Given", datatype=XSD.string) ))
g.add((URIRef(ont+'has_score_given'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'has_score_given'), URIRef(rdfs+'range'), XSD.integer ))


g.add((URIRef(ont+'has_type'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_type'), URIRef(rdfs+'label'), Literal("Has Type", datatype=XSD.string) ))
g.add((URIRef(ont+'has_type'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'has_type'), URIRef(rdfs+'range'), URIRef(cct+'Pitcher') ))
g.add((URIRef(ont+'has_type'), URIRef(rdfs+'range'), URIRef(cct+'Batter') ))

g.add((URIRef(ont+'has_status'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_status'), URIRef(rdfs+'label'), Literal("Has Status", datatype=XSD.string) ))
g.add((URIRef(ont+'has_status'), URIRef(rdfs+'domain'), URIRef(ppl+'player') ))
g.add((URIRef(ont+'has_status'), URIRef(rdfs+'range'), URIRef(cct+'player'+'_overall') ))

g.add((URIRef(ont+'has_W'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_W'), URIRef(rdfs+'label'), Literal("Has W", datatype=XSD.string) ))
g.add((URIRef(ont+'has_W'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_W'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_L'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_L'), URIRef(rdfs+'label'), Literal("Has L", datatype=XSD.string) ))
g.add((URIRef(ont+'has_L'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_L'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_ERA'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_ERA'), URIRef(rdfs+'label'), Literal("Has ERA", datatype=XSD.string) ))
g.add((URIRef(ont+'has_ERA'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_ERA'), URIRef(rdfs+'range'), XSD.float ))

g.add((URIRef(ont+'has_G'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_G'), URIRef(rdfs+'label'), Literal("Has G", datatype=XSD.string) ))
g.add((URIRef(ont+'has_G'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_G'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_GS'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_GS'), URIRef(rdfs+'label'), Literal("Has GS", datatype=XSD.string) ))
g.add((URIRef(ont+'has_GS'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_GS'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_SV'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_SV'), URIRef(rdfs+'label'), Literal("Has SV", datatype=XSD.string) ))
g.add((URIRef(ont+'has_SV'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_SV'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_IP'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'label'), Literal("Has IP", datatype=XSD.string) ))
g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'range'), XSD.float ))

g.add((URIRef(ont+'has_SO'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'label'), Literal("Has SO", datatype=XSD.string) ))
g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_WHIP'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_WHIP'), URIRef(rdfs+'label'), Literal("Has WHIP", datatype=XSD.string) ))
g.add((URIRef(ont+'has_WHIP'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_overall') ))
g.add((URIRef(ont+'has_WHIP'), URIRef(rdfs+'range'), XSD.float ))


g.add((URIRef(ont+'has_PA'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_PA'), URIRef(rdfs+'label'), Literal("Has PA", datatype=XSD.string) ))
g.add((URIRef(ont+'has_PA'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_PA'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_AB'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'label'), Literal("Has AB", datatype=XSD.string) ))
g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_R'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'label'), Literal("Has R", datatype=XSD.string) ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_H'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'label'), Literal("Has H", datatype=XSD.string) ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_HR'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_HR'), URIRef(rdfs+'label'), Literal("Has HR", datatype=XSD.string) ))
g.add((URIRef(ont+'has_HR'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_HR'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_RBI'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'label'), Literal("Has RBI", datatype=XSD.string) ))
g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_SB'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_SB'), URIRef(rdfs+'label'), Literal("Has SB", datatype=XSD.string) ))
g.add((URIRef(ont+'has_SB'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_SB'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_AVG'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_AVG'), URIRef(rdfs+'label'), Literal("Has AVG", datatype=XSD.string) ))
g.add((URIRef(ont+'has_AVG'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_AVG'), URIRef(rdfs+'range'), XSD.float ))

g.add((URIRef(ont+'has_OBP'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_OBP'), URIRef(rdfs+'label'), Literal("Has OBP", datatype=XSD.string) ))
g.add((URIRef(ont+'has_OBP'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_OBP'), URIRef(rdfs+'range'), XSD.float ))

g.add((URIRef(ont+'has_SLG'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_SLG'), URIRef(rdfs+'label'), Literal("Has SLG", datatype=XSD.string) ))
g.add((URIRef(ont+'has_SLG'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_SLG'), URIRef(rdfs+'range'), XSD.float ))

g.add((URIRef(ont+'has_OPS'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_OPS'), URIRef(rdfs+'label'), Literal("Has OPS", datatype=XSD.string) ))
g.add((URIRef(ont+'has_OPS'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_overall') ))
g.add((URIRef(ont+'has_OPS'), URIRef(rdfs+'range'), XSD.float ))


split_performance = ['monthly','game_type','platoon','inning']
for i in range(0, 4):
    g.add((URIRef(ont+'has_status'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_status'), URIRef(rdfs+'label'), Literal("Has Status", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_status'), URIRef(rdfs+'domain'), URIRef(cct+'player'+'_split') ))
    g.add((URIRef(ont+'has_status'), URIRef(rdfs+'range'), URIRef(cct+'player'+'_'+split_performance[i]) ))


    g.add((URIRef(ont+'has_W'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_W'), URIRef(rdfs+'label'), Literal("Has W", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_W'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_W'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_L'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_L'), URIRef(rdfs+'label'), Literal("Has L", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_L'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_L'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_ERA'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_ERA'), URIRef(rdfs+'label'), Literal("Has ERA", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_ERA'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_ERA'), URIRef(rdfs+'range'), XSD.float ))

    g.add((URIRef(ont+'has_G'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_G'), URIRef(rdfs+'label'), Literal("Has G", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_G'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_G'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_GS'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_GS'), URIRef(rdfs+'label'), Literal("Has GS", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_GS'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_GS'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_SV'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_SV'), URIRef(rdfs+'label'), Literal("Has SV", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_SV'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_SV'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_IP'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'label'), Literal("Has IP", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'range'), XSD.float ))

    g.add((URIRef(ont+'has_BF'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_BF'), URIRef(rdfs+'label'), Literal("Has BF", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_BF'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_BF'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_H'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_H'), URIRef(rdfs+'label'), Literal("Has H", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_H'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_H'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_R'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_R'), URIRef(rdfs+'label'), Literal("Has R", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_R'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_R'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_ER'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_ER'), URIRef(rdfs+'label'), Literal("Has ER", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_ER'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_ER'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_BB'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_BB'), URIRef(rdfs+'label'), Literal("Has BB", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_BB'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_BB'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_SO'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'label'), Literal("Has SO", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_WHIP'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_WHIP'), URIRef(rdfs+'label'), Literal("Has WHIP", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_WHIP'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_WHIP'), URIRef(rdfs+'range'), XSD.float ))


    g.add((URIRef(ont+'has_PA'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_PA'), URIRef(rdfs+'label'), Literal("Has PA", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_PA'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_PA'), URIRef(rdfs+'range'), XSD.integer ))

    g.add((URIRef(ont+'has_AB'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'label'), Literal("Has AB", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_R'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_R'), URIRef(rdfs+'label'), Literal("Has R", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_R'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_R'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_H'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_H'), URIRef(rdfs+'label'), Literal("Has H", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_H'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_H'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_2B'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_2B'), URIRef(rdfs+'label'), Literal("Has 2B", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_2B'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_2B'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_3B'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_3B'), URIRef(rdfs+'label'), Literal("Has 3B", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_3B'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_3B'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_HR'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_HR'), URIRef(rdfs+'label'), Literal("Has HR", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_HR'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_HR'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_RBI'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'label'), Literal("Has RBI", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_BB'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_BB'), URIRef(rdfs+'label'), Literal("Has BB", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_BB'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_BB'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_SO'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'label'), Literal("Has SO", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_SO'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_SB'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_SB'), URIRef(rdfs+'label'), Literal("Has SB", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_SB'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_SB'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_CS'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_CS'), URIRef(rdfs+'label'), Literal("Has CS", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_CS'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_CS'), URIRef(rdfs+'range'), XSD.integer ))
    
    g.add((URIRef(ont+'has_HBP'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_HBP'), URIRef(rdfs+'label'), Literal("Has HBP", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_HBP'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_HBP'), URIRef(rdfs+'range'), XSD.float ))
    
    g.add((URIRef(ont+'has_AVG'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_AVG'), URIRef(rdfs+'label'), Literal("Has AVG", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_AVG'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_AVG'), URIRef(rdfs+'range'), XSD.float ))
    
    g.add((URIRef(ont+'has_OBP'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_OBP'), URIRef(rdfs+'label'), Literal("Has OBP", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_OBP'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_OBP'), URIRef(rdfs+'range'), XSD.float ))
    
    g.add((URIRef(ont+'has_SLG'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_SLG'), URIRef(rdfs+'label'), Literal("Has SLG", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_SLG'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_SLG'), URIRef(rdfs+'range'), XSD.float ))
    
    g.add((URIRef(ont+'has_OPS'), RDF.type, RDF.Property ))
    g.add((URIRef(ont+'has_OPS'), URIRef(rdfs+'label'), Literal("Has OPS", datatype=XSD.string) ))
    g.add((URIRef(ont+'has_OPS'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+split_performance[i]) ))
    g.add((URIRef(ont+'has_OPS'), URIRef(rdfs+'range'), XSD.float ))


g.add((URIRef(ont+'for_match'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'for_match'), URIRef(rdfs+'label'), Literal("For Match", datatype=XSD.string) ))
g.add((URIRef(ont+'for_match'), URIRef(rdfs+'domain'), URIRef(cct+'player'+'_match') ))
g.add((URIRef(ont+'for_match'), URIRef(rdfs+'range'), URIRef(mmc+'match') ))

g.add((URIRef(ont+'has_player_played'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_player_played'), URIRef(rdfs+'label'), Literal("Has Player Played", datatype=XSD.string) ))
g.add((URIRef(ont+'has_player_played'), URIRef(rdfs+'domain'), URIRef(mmc+'match') ))
g.add((URIRef(ont+'has_player_played'), URIRef(rdfs+'range'), URIRef(ppl+'player') ))

g.add((URIRef(ont+'has_status_for_match'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_status_for_match'), URIRef(rdfs+'label'), Literal("Has Status for Match", datatype=XSD.string) ))
g.add((URIRef(ont+'has_status_for_match'), URIRef(rdfs+'domain'), URIRef(cct+'player'+'_'+'match') ))
g.add((URIRef(ont+'has_status_for_match'), URIRef(rdfs+'range'), URIRef(mmc+'match') ))


g.add((URIRef(ont+'has_IP'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'label'), Literal("Has IP", datatype=XSD.string) ))
g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+'match') ))
g.add((URIRef(ont+'has_IP'), URIRef(rdfs+'range'), XSD.float ))

g.add((URIRef(ont+'has_H'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'label'), Literal("Has H", datatype=XSD.string) ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+'match') ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_R'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'label'), Literal("Has R", datatype=XSD.string) ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+'match') ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_ER'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_ER'), URIRef(rdfs+'label'), Literal("Has ER", datatype=XSD.string) ))
g.add((URIRef(ont+'has_ER'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+'match') ))
g.add((URIRef(ont+'has_ER'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'inference-has_ER'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'inference-has_ER'), URIRef(rdfs+'label'), Literal("Inference- Has ER", datatype=XSD.string) ))
g.add((URIRef(ont+'inference-has_ER'), URIRef(rdfs+'domain'), URIRef(cct+'Pitcher'+'_'+'match') ))
g.add((URIRef(ont+'inference-has_ER'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_AB'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'label'), Literal("Has AB", datatype=XSD.string) ))
g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+'match') ))
g.add((URIRef(ont+'has_AB'), URIRef(rdfs+'range'), XSD.float ))

g.add((URIRef(ont+'has_R'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'label'), Literal("Has R", datatype=XSD.string) ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+'match') ))
g.add((URIRef(ont+'has_R'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_H'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'label'), Literal("Has H", datatype=XSD.string) ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+'match') ))
g.add((URIRef(ont+'has_H'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'has_RBI'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'label'), Literal("Has RBI", datatype=XSD.string) ))
g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+'match') ))
g.add((URIRef(ont+'has_RBI'), URIRef(rdfs+'range'), XSD.integer ))

g.add((URIRef(ont+'inference-has_RBI'), RDF.type, RDF.Property ))
g.add((URIRef(ont+'inference-has_RBI'), URIRef(rdfs+'label'), Literal("Inference- Has RBI", datatype=XSD.string) ))
g.add((URIRef(ont+'inference-has_RBI'), URIRef(rdfs+'domain'), URIRef(cct+'Batter'+'_'+'match') ))
g.add((URIRef(ont+'inference-has_RBI'), URIRef(rdfs+'range'), XSD.integer ))






g.add((URIRef(ttm+'team'), RDF.type, FOAF.Group ))
g.add((URIRef(ttm+'team'), URIRef(schema+'name'), XSD.string ))
g.add((URIRef(ttm+'team'), URIRef(ont+'has_match'), URIRef(cct+'team'+'_match') ))
g.add((URIRef(cct+'team'+'_match'), RDF.type, URIRef(ccs+'Cluster') ))
g.add((URIRef(ttm+'team'), URIRef(ont+'has_player'), URIRef(cct+'team'+'_player') ))
g.add((URIRef(cct+'team'+'_player'), RDF.type, URIRef(ccs+'Cluster') ))


g.add((URIRef(ppl+'player'), RDF.type, FOAF.Person))
g.add((URIRef(ppl+'player'), URIRef(schema+'name'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'uri'), XSD.anyURI ))
g.add((URIRef(ppl+'player'), URIRef(ont+'url'), XSD.anyURI ))
g.add((URIRef(ppl+'player'), URIRef(ont+'belong_to_team'), URIRef(ttm+'team') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'position'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'pos_bat'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'pos_throw'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_split') ))
g.add((URIRef(cct+'player'+'_split'), RDF.type, URIRef(ccs+'Cluster') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_match'), URIRef(cct+'player'+'_match') ))
g.add((URIRef(cct+'player'+'_match'), RDF.type, URIRef(ccs+'Cluster') ))
g.add((URIRef(cct+'team'+'_player'), URIRef(ont+'for_player'), (URIRef(ppl+'player')) ))


#g.add((URIRef(ttm+this_row_team), URIRef(ont+'has_match'), URIRef(mmc+str(row['GeneralInfo']).replace(' ','_')) ))
g.add((URIRef(cct+'team'+'_match'), URIRef(ont+'for_match'), URIRef(mmc+'match') ))
g.add((URIRef(mmc+'match'), RDF.type, URIRef(ccs+'Match') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_by'), URIRef(ttm+'team') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_on'), XSD.date ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_against'), URIRef(ttm+'team') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'ends_with'), XSD.string ))
g.add((URIRef(mmc+'match'), URIRef(ont+'inference-ends_with'), XSD.string ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_at'), XSD.string ))
g.add((URIRef(mmc+'match'), URIRef(ont+'has_score_secured'), XSD.integer ))
g.add((URIRef(mmc+'match'), URIRef(ont+'has_score_given'), XSD.integer ))



g.add((URIRef(ppl+'player'), URIRef(ont+'has_type'), URIRef(cct+'Pitcher') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_type'), URIRef(cct+'Batter') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_overall') ))
g.add((URIRef(cct+'player'+'_overall'), RDF.type, URIRef(ccs+'Cluster') ))

g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_overall') ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_overall') ))

g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_W'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_L'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_ERA'), XSD.float ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_G'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_GS'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_SV'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_IP'), XSD.float ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_SO'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_WHIP'), XSD.float ))

g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_PA'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_AB'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_R'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_H'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_HR'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_RBI'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_SB'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_AVG'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_OBP'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_SLG'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_OPS'), XSD.float ))



split_performance = ['monthly','game_type','platoon','inning']
for i in range(0, 4):
    g.add((URIRef(cct+'player'+'_split'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_'+split_performance[i]) ))
    g.add((URIRef(cct+'player'+'_'+split_performance[i]), RDF.type, URIRef(ccs+'Cluster') ))

    g.add((URIRef(cct+'Pitcher_'+split_performance[i]), URIRef(rdfs+'subClassOf'), URIRef(cct+'player_'+split_performance[i]) ))
    g.add((URIRef(cct+'Batter_'+split_performance[i]), URIRef(rdfs+'subClassOf'), URIRef(cct+'player_'+split_performance[i]) ))

    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_W'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_L'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_ERA'), XSD.float ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_G'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_GS'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_SV'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_IP'), XSD.float ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_BF'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_H'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_R'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_ER'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_BB'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_SO'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_WHIP'), XSD.float ))

    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_PA'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_AB'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_R'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_H'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_2B'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_3B'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_HR'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_RBI'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_BB'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_SO'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_SB'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_CS'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_HBP'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_AVG'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_OBP'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_SLG'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_OPS'), XSD.float ))



g.add((URIRef(cct+'player'+'_match'), URIRef(ont+'for_match'), URIRef(mmc+'match') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'has_player_played'), URIRef(ppl+'player') ))

#g.add((URIRef(cct+'player'+'_match'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_'+'match') ))
#g.add((URIRef(cct+'player'+'_match'), RDF.type, URIRef(ccs+'Cluster') ))

g.add((URIRef(cct+'player'+'_'+'match'), URIRef(ont+'has_status_for_match'), URIRef(mmc+'match') ))

g.add((URIRef(cct+'Pitcher'+'_match'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_match') ))
g.add((URIRef(cct+'Batter'+'_match'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_match') ))

g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_IP'), XSD.float ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_H'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_R'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_ER'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'inference-has_ER'), XSD.integer ))

g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_AB'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_R'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_H'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_RBI'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'inference-has_RBI'), XSD.integer ))



#print(g.serialize(format='turtle'))
g.serialize('ontology.ttl',format='turtle')

<Graph identifier=N0bbdf49d660e4a9097a705239ea6d3ae (<class 'rdflib.graph.Graph'>)>

In [32]:
g = Graph()
ppl = Namespace('http://dsci558.org/player/')
loc = Namespace('http://mylocations.org/addresses/')
schema = Namespace('http://schema.org/')
ttm = Namespace('http://dsci558.org/team/')
mmc = Namespace('http://dsci558.org/match/')
ccs = Namespace('http://dsci558.org/class/')
cct = Namespace('http://dsci558.org/cluster/')
ont = Namespace('http://dsci558.org/ontology/')
rdf0 = Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')

g.bind('bkgpy', ppl)
g.bind('bkgtm', ttm)
g.bind('bkgmc', mmc)
g.bind('bkgcs', ccs)
g.bind('bkgct', cct)
g.bind('bkgot', ont)
g.bind('rdf', rdf0)
g.bind('rdfs', rdfs)
g.bind('foaf', FOAF)
g.bind('xsd', XSD)

g.add((URIRef(ccs+'Cluster'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(ccs+'Match'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(ttm+'team'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(ppl+'player'), RDF.type, URIRef(rdfs+'Class') ))
g.add((URIRef(mmc+'match'), RDF.type, URIRef(rdfs+'Class') ))


g.add((URIRef(ttm+'team'), RDF.type, FOAF.Group ))
g.add((URIRef(ttm+'team'), URIRef(schema+'name'), XSD.string ))
g.add((URIRef(ttm+'team'), URIRef(ont+'has_match'), URIRef(cct+'team'+'_match') ))
g.add((URIRef(cct+'team'+'_match'), RDF.type, URIRef(ccs+'Cluster') ))
g.add((URIRef(ttm+'team'), URIRef(ont+'has_player'), URIRef(cct+'team'+'_player') ))
g.add((URIRef(cct+'team'+'_player'), RDF.type, URIRef(ccs+'Cluster') ))


g.add((URIRef(ppl+'player'), RDF.type, FOAF.Person))
g.add((URIRef(ppl+'player'), URIRef(schema+'name'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'uri'), XSD.anyURI ))
g.add((URIRef(ppl+'player'), URIRef(ont+'url'), XSD.anyURI ))
g.add((URIRef(ppl+'player'), URIRef(ont+'belong_to_team'), URIRef(ttm+'team') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'position'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'pos_bat'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'pos_throw'), XSD.string ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_split') ))
g.add((URIRef(cct+'player'+'_split'), RDF.type, URIRef(ccs+'Cluster') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_match'), URIRef(cct+'player'+'_match') ))
g.add((URIRef(cct+'player'+'_match'), RDF.type, URIRef(ccs+'Cluster') ))
g.add((URIRef(cct+'team'+'_player'), URIRef(ont+'for_player'), (URIRef(ppl+'player')) ))


#g.add((URIRef(ttm+this_row_team), URIRef(ont+'has_match'), URIRef(mmc+str(row['GeneralInfo']).replace(' ','_')) ))
g.add((URIRef(cct+'team'+'_match'), URIRef(ont+'for_match'), URIRef(mmc+'match') ))
g.add((URIRef(mmc+'match'), RDF.type, URIRef(ccs+'Match') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_by'), URIRef(ttm+'team') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_on'), XSD.date ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_against'), URIRef(ttm+'team') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'ends_with'), XSD.string ))
g.add((URIRef(mmc+'match'), URIRef(ont+'inference-ends_with'), XSD.string ))
g.add((URIRef(mmc+'match'), URIRef(ont+'plays_at'), XSD.string ))
g.add((URIRef(mmc+'match'), URIRef(ont+'has_score_secured'), XSD.integer ))
g.add((URIRef(mmc+'match'), URIRef(ont+'has_score_given'), XSD.integer ))



g.add((URIRef(ppl+'player'), URIRef(ont+'has_type'), URIRef(cct+'Pitcher') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_type'), URIRef(cct+'Batter') ))
g.add((URIRef(ppl+'player'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_overall') ))
g.add((URIRef(cct+'player'+'_overall'), RDF.type, URIRef(ccs+'Cluster') ))

g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_overall') ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_overall') ))

g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_W'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_L'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_ERA'), XSD.float ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_G'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_GS'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_SV'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_IP'), XSD.float ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_SO'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_overall'), URIRef(ont+'has_WHIP'), XSD.float ))

g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_PA'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_AB'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_R'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_H'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_HR'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_RBI'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_SB'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_AVG'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_OBP'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_SLG'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_overall'), URIRef(ont+'has_OPS'), XSD.float ))



split_performance = ['monthly','game_type','platoon','inning']
for i in range(0, 1):
    g.add((URIRef(cct+'player'+'_split'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_'+split_performance[i]) ))
    g.add((URIRef(cct+'player'+'_'+split_performance[i]), RDF.type, URIRef(ccs+'Cluster') ))

    g.add((URIRef(cct+'Pitcher_'+split_performance[i]), URIRef(rdfs+'subClassOf'), URIRef(cct+'player_'+split_performance[i]) ))
    g.add((URIRef(cct+'Batter_'+split_performance[i]), URIRef(rdfs+'subClassOf'), URIRef(cct+'player_'+split_performance[i]) ))

    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_W'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_L'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_ERA'), XSD.float ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_G'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_GS'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_SV'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_IP'), XSD.float ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_BF'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_H'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_R'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_ER'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_BB'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_SO'), XSD.integer ))
    g.add((URIRef(cct+'Pitcher'+'_'+split_performance[i]), URIRef(ont+'has_WHIP'), XSD.float ))

    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_PA'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_AB'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_R'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_H'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_2B'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_3B'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_HR'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_RBI'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_BB'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_SO'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_SB'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_CS'), XSD.integer ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_HBP'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_AVG'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_OBP'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_SLG'), XSD.float ))
    g.add((URIRef(cct+'Batter'+'_'+split_performance[i]), URIRef(ont+'has_OPS'), XSD.float ))



g.add((URIRef(cct+'player'+'_match'), URIRef(ont+'for_match'), URIRef(mmc+'match') ))
g.add((URIRef(mmc+'match'), URIRef(ont+'has_player_played'), URIRef(ppl+'player') ))

#g.add((URIRef(cct+'player'+'_match'), URIRef(ont+'has_status'), URIRef(cct+'player'+'_'+'match') ))
#g.add((URIRef(cct+'player'+'_match'), RDF.type, URIRef(ccs+'Cluster') ))

g.add((URIRef(cct+'player'+'_'+'match'), URIRef(ont+'has_status_for_match'), URIRef(mmc+'match') ))

g.add((URIRef(cct+'Pitcher'+'_match'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_match') ))
g.add((URIRef(cct+'Batter'+'_match'), URIRef(rdfs+'subClassOf'), URIRef(cct+'player'+'_match') ))

g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_IP'), XSD.float ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_H'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_R'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'has_ER'), XSD.integer ))
g.add((URIRef(cct+'Pitcher'+'_'+'match'), URIRef(ont+'inference-has_ER'), XSD.integer ))

g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_AB'), XSD.float ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_R'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_H'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'has_RBI'), XSD.integer ))
g.add((URIRef(cct+'Batter'+'_'+'match'), URIRef(ont+'inference-has_RBI'), XSD.integer ))



#print(g.serialize(format='turtle'))
g.serialize('ontology_simplified_for_visualization.ttl',format='turtle')

<Graph identifier=Na99524d39045491ea332f28accb59747 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# For Ontology Visualization
# refer to https://service.tib.eu/webvowl/#file=ontology.ttl
# refer to https://www.ldf.fi/service/rdf-grapher

# https://service.tib.eu/webvowl/#opts=doc=0;#file=ontology.ttl
'''
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

gg = rdflib.Graph()
result = gg.parse('ontology.ttl', format='turtle')
G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=1)
edge_labels = nx.get_edge_attributes(G, 'r')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
nx.draw(G, with_labels=True)

#if not in interactive mode for 
plt.show()
'''

In [13]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

query = """
SELECT ?team_name ?match ?object1 ?object2name
WHERE {
    ?team <http://dsci558.org/ontology/has_player> ?player_cluster.
    ?team <http://schema.org/name> ?teamname.
    ?team <http://dsci558.org/ontology/has_match> ?match_cluster.
    ?match_cluster <http://dsci558.org/ontology/for_match> ?match.
    ?match <http://dsci558.org/ontology/plays_at> ?object1.
    ?match <http://dsci558.org/ontology/plays_by> ?object2.
    ?object2 <http://schema.org/name> ?object2name.
}
"""

qres = gg.query(query)
for row in qres:
    #print("%s" % row)
    #print(row)
    #print(f"{row.teamname} {row.match} {row.object1}")
    print(row.object1)

AWAY
AWAY
HOME
AWAY
HOME
AWAY
HOME
HOME
AWAY
HOME
AWAY
AWAY
AWAY
HOME
HOME
AWAY
AWAY
AWAY
AWAY
HOME
HOME
HOME
HOME
AWAY
HOME
HOME
HOME
HOME
AWAY
AWAY
AWAY
AWAY
HOME
AWAY
AWAY
HOME
HOME
HOME
HOME
AWAY
AWAY
HOME
HOME
AWAY
AWAY
HOME
HOME
AWAY
HOME
HOME
HOME
AWAY
HOME
AWAY
HOME
HOME
HOME
HOME
HOME
HOME
HOME
AWAY
HOME
HOME
AWAY
HOME
HOME
HOME
HOME
HOME
AWAY
HOME
HOME
AWAY
AWAY
AWAY
HOME
HOME
HOME
AWAY
AWAY
HOME
AWAY
HOME
HOME
AWAY
AWAY
AWAY
HOME
HOME
AWAY
HOME
HOME
AWAY
AWAY
HOME
HOME
HOME
HOME
HOME
HOME
HOME
AWAY
AWAY
HOME
HOME
HOME
HOME
HOME
HOME
HOME
HOME
HOME
HOME
AWAY
AWAY
AWAY
AWAY
AWAY
AWAY
AWAY
AWAY
AWAY
HOME
HOME
AWAY
HOME
HOME
HOME
AWAY
AWAY
AWAY
AWAY
AWAY
HOME
AWAY
AWAY
AWAY
HOME
AWAY
HOME
HOME
AWAY
HOME
AWAY
AWAY
AWAY
AWAY
AWAY
HOME
AWAY
HOME
HOME
HOME
AWAY
HOME
HOME
HOME
HOME
HOME
HOME
AWAY
HOME
AWAY
AWAY
HOME
AWAY
AWAY
AWAY
HOME
HOME
AWAY
AWAY
AWAY
AWAY
HOME
AWAY
AWAY
AWAY
AWAY
HOME
AWAY
HOME
HOME
HOME
AWAY
HOME
AWAY
AWAY
AWAY
HOME
HOME
AWAY
AWAY
AWAY
HOME
HOME
HOME
AWAY
HOME


In [14]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Player belongs to which team?
query = """
SELECT ?player_name ?team_name
WHERE {
    ?player <http://dsci558.org/ontology/belong_to_team> ?team.
    ?player <http://schema.org/name> ?player_name.
    ?team <http://schema.org/name> ?team_name.
}
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.player_name, '-(belongs to team)->', row.team_name)

Cole Irvin -(belongs to team)-> Oakland Athletics
Seth Brown -(belongs to team)-> Oakland Athletics
Tony Kemp -(belongs to team)-> Oakland Athletics
Elvis Andrus -(belongs to team)-> Oakland Athletics
Sean Murphy -(belongs to team)-> Oakland Athletics
Sandy Alcantara -(belongs to team)-> Miami Marlins
Pablo Lopez -(belongs to team)-> Miami Marlins
Miguel Rojas -(belongs to team)-> Miami Marlins
Jesus Aguilar -(belongs to team)-> Miami Marlins
Merrill Kelly -(belongs to team)-> Arizona Diamondbacks
Zac Gallen -(belongs to team)-> Arizona Diamondbacks
Josh Rojas -(belongs to team)-> Arizona Diamondbacks
Ketel Marte -(belongs to team)-> Arizona Diamondbacks
Daulton Varsho -(belongs to team)-> Arizona Diamondbacks
Christian Walker -(belongs to team)-> Arizona Diamondbacks
Jose Quintana -(belongs to team)-> St. Louis Cardinals
Adam Wainwright -(belongs to team)-> St. Louis Cardinals
Miles Mikolas -(belongs to team)-> St. Louis Cardinals
Jordan Montgomery -(belongs to team)-> St. Louis Cardi

In [15]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Team ever plays against another team?
query = """
SELECT DISTINCT ?team1_name ?team2_name
WHERE {
    ?match <http://dsci558.org/ontology/plays_by> ?team1.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.team1_name, '-(plays against)->', row.team2_name)

Miami Marlins -(plays against)-> Milwaukee Brewers
Cincinnati Reds -(plays against)-> Milwaukee Brewers
Pittsburgh Pirates -(plays against)-> Milwaukee Brewers
Chicago Cubs -(plays against)-> Milwaukee Brewers
Minnesota Twins -(plays against)-> Milwaukee Brewers
St. Louis Cardinals -(plays against)-> Milwaukee Brewers
San Francisco Giants -(plays against)-> Milwaukee Brewers
Boston Red Sox -(plays against)-> Milwaukee Brewers
New York Yankees -(plays against)-> Milwaukee Brewers
Los Angeles Dodgers -(plays against)-> Milwaukee Brewers
Philadelphia Phillies -(plays against)-> Milwaukee Brewers
New York Mets -(plays against)-> Milwaukee Brewers
Atlanta Braves -(plays against)-> Milwaukee Brewers
San Diego Padres -(plays against)-> Milwaukee Brewers
Detroit Tigers -(plays against)-> San Francisco Giants
Chicago Cubs -(plays against)-> San Francisco Giants
Minnesota Twins -(plays against)-> San Francisco Giants
St. Louis Cardinals -(plays against)-> San Francisco Giants
Arizona Diamondback

In [16]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Team plays against another team?
query = """
SELECT DISTINCT ?team1_name ?team2_name ?date ?location ?simple_result ?score_secured ?score_given
WHERE {
    ?team1 <http://dsci558.org/ontology/has_player> ?player_cluster.
    ?team1 <http://dsci558.org/ontology/has_match> ?match_cluster.
    ?match_cluster <http://dsci558.org/ontology/for_match> ?match.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/ends_with> ?simple_result.
    ?match <http://dsci558.org/ontology/has_score_secured> ?score_secured.
    ?match <http://dsci558.org/ontology/has_score_given> ?score_given.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.
    
    
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.team1_name, '-(plays against)->', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location, '-(ends with)->', row.simple_result, '(', row.score_secured, ':', row.score_given, ')')

Miami Marlins -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-09-29 -(plays at)-> AWAY -(ends with)-> WIN ( 4 : 2 )
Cincinnati Reds -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-09-25 -(plays at)-> HOME -(ends with)-> WIN ( 2 : 1 )
Cincinnati Reds -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-08-06 -(plays at)-> AWAY -(ends with)-> WIN ( 7 : 5 )
Pittsburgh Pirates -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-08-04 -(plays at)-> HOME -(ends with)-> WIN ( 5 : 4 )
Pittsburgh Pirates -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-08-31 -(plays at)-> AWAY -(ends with)-> LOSE ( 1 : 6 )
Chicago Cubs -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-04-07 -(plays at)-> HOME -(ends with)-> WIN ( 5 : 4 )
Cincinnati Reds -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-05-11 -(plays at)-> HOME -(ends with)-> WIN ( 14 : 11 )
Chicago Cubs -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-05-30 -(plays at)-> HOME -(ends with)-> LOSE 

In [17]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Prediction - Game result
query = """
SELECT DISTINCT ?team1_name ?team2_name ?date ?location ?simple_result ?inference_simple_result
WHERE {
    ?team1 <http://dsci558.org/ontology/has_player> ?player_cluster.
    ?team1 <http://dsci558.org/ontology/has_match> ?match_cluster.
    ?match_cluster <http://dsci558.org/ontology/for_match> ?match.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/ends_with> ?simple_result.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.
    ?match <http://dsci558.org/ontology/inference-ends_with> ?inference_simple_result.
    
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
LIMIT 10
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.team1_name, '-(plays against)->', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location, '-(ends with)->', row.simple_result, '<-(prediction)-', row.inference_simple_result)

Miami Marlins -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-09-29 -(plays at)-> AWAY -(ends with)-> WIN <-(inference)- WIN
Cincinnati Reds -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-09-25 -(plays at)-> HOME -(ends with)-> WIN <-(inference)- LOSE
Cincinnati Reds -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-08-06 -(plays at)-> AWAY -(ends with)-> WIN <-(inference)- WIN
Pittsburgh Pirates -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-08-04 -(plays at)-> HOME -(ends with)-> WIN <-(inference)- WIN
Pittsburgh Pirates -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-08-31 -(plays at)-> AWAY -(ends with)-> LOSE <-(inference)- LOSE
Cincinnati Reds -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-05-11 -(plays at)-> HOME -(ends with)-> WIN <-(inference)- WIN
Chicago Cubs -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-05-30 -(plays at)-> HOME -(ends with)-> LOSE <-(inference)- WIN
Minnesota Twins -(plays against)-> Milwaukee Brewe

In [18]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Player plays in a match?
query = """
SELECT DISTINCT ?player_name ?team1_name ?team2_name ?date ?location
WHERE {
    ?team1 a <http://xmlns.com/foaf/0.1/Group>.
    ?team2 a <http://xmlns.com/foaf/0.1/Group>.
    ?player a <http://xmlns.com/foaf/0.1/Person>.

    ?team1 <http://dsci558.org/ontology/has_player> ?player_cluster.
    ?player_cluster <http://dsci558.org/ontology/for_player> ?player.
    ?player <http://dsci558.org/ontology/has_match> ?match_cluster.
    ?match_cluster <http://dsci558.org/ontology/for_match> ?match.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.
    
    ?player <http://schema.org/name> ?player_name.
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
LIMIT 10
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.player_name, '-(belongs to)->',row.team1_name, '-(plays against)->', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location)

Cole Irvin -(belongs to)-> Oakland Athletics -(plays against)-> Miami Marlins -(plays on)-> 2022-08-24 -(plays at)-> HOME
Sandy Alcantara -(belongs to)-> Miami Marlins -(plays against)-> New York Mets -(plays on)-> 2022-06-24 -(plays at)-> HOME
Sandy Alcantara -(belongs to)-> Miami Marlins -(plays against)-> New York Mets -(plays on)-> 2022-07-10 -(plays at)-> AWAY
Merrill Kelly -(belongs to)-> Arizona Diamondbacks -(plays against)-> San Francisco Giants -(plays on)-> 2022-08-16 -(plays at)-> AWAY
Merrill Kelly -(belongs to)-> Arizona Diamondbacks -(plays against)-> Houston Astros -(plays on)-> 2022-04-13 -(plays at)-> HOME
Merrill Kelly -(belongs to)-> Arizona Diamondbacks -(plays against)-> Los Angeles Dodgers -(plays on)-> 2022-05-28 -(plays at)-> HOME
Pablo Lopez -(belongs to)-> Miami Marlins -(plays against)-> St. Louis Cardinals -(plays on)-> 2022-04-21 -(plays at)-> HOME
Pablo Lopez -(belongs to)-> Miami Marlins -(plays against)-> Chicago Cubs -(plays on)-> 2022-08-06 -(plays at

In [19]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Player plays against another player in a match?
query = """
SELECT DISTINCT ?match ?player1_name ?player2_name ?team1_name ?team2_name ?date ?location
WHERE {
    ?match a <http://dsci558.org/class/Match>.

    ?match <http://dsci558.org/ontology/has_player_played> ?player1.
    ?match <http://dsci558.org/ontology/has_player_played> ?player2.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/plays_by> ?team1.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.
    
    ?team1 <http://dsci558.org/ontology/has_player> ?player1_cluster.
    ?player1_cluster <http://dsci558.org/ontology/for_player> ?player1.
    ?team2 <http://dsci558.org/ontology/has_player> ?player2_cluster.
    ?player2_cluster <http://dsci558.org/ontology/for_player> ?player2.

    ?player1 <http://schema.org/name> ?player1_name.
    ?player2 <http://schema.org/name> ?player2_name.
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
LIMIT 10
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.player1_name, '<-(from)-',row.team1_name, '-(plays against)->', row.player2_name, '<-(from)-', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location)

Jose Abreu <-(from)- Chicago White Sox -(plays against)-> Elvis Andrus <-(from)- Oakland Athletics -(plays on)-> 2022-09-10 -(plays at)-> AWAY
AJ Pollock <-(from)- Chicago White Sox -(plays against)-> Elvis Andrus <-(from)- Oakland Athletics -(plays on)-> 2022-09-10 -(plays at)-> AWAY
Andrew Vaughn <-(from)- Chicago White Sox -(plays against)-> Elvis Andrus <-(from)- Oakland Athletics -(plays on)-> 2022-09-10 -(plays at)-> AWAY
Andrew Benintendi <-(from)- Kansas City Royals -(plays against)-> Whit Merrifield <-(from)- Toronto Blue Jays -(plays on)-> 2022-06-08 -(plays at)-> HOME
MJ Melendez <-(from)- Kansas City Royals -(plays against)-> Whit Merrifield <-(from)- Toronto Blue Jays -(plays on)-> 2022-06-08 -(plays at)-> HOME
Bobby Witt Jr. <-(from)- Kansas City Royals -(plays against)-> Whit Merrifield <-(from)- Toronto Blue Jays -(plays on)-> 2022-06-08 -(plays at)-> HOME
Yuli Gurriel <-(from)- Houston Astros -(plays against)-> Trey Mancini <-(from)- Baltimore Orioles -(plays on)-> 202

In [25]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# How player (a hitter) performs in a match
query = """
SELECT DISTINCT ?match ?player1_name ?player_H_in_match ?player_RBI_in_match ?team1_name ?team2_name ?date ?location
WHERE {
    ?match a <http://dsci558.org/class/Match>.

    ?match <http://dsci558.org/ontology/has_player_played> ?player1.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/plays_by> ?team1.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.

    ?player1 <http://dsci558.org/ontology/has_match> ?player_match.
    ?player_match <http://dsci558.org/ontology/has_status> ?player_status_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_status_for_match> ?match.
    ?player_status_in_match <http://dsci558.org/ontology/has_H> ?player_H_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_RBI> ?player_RBI_in_match.
    
    ?team1 <http://dsci558.org/ontology/has_player> ?player1_cluster.
    ?player1_cluster <http://dsci558.org/ontology/for_player> ?player1.

    ?player1 <http://schema.org/name> ?player1_name.
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
LIMIT 10
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.player1_name, '<-(from)-',row.team1_name, '-(has H in match)', row.player_H_in_match , '-(has RBI in match)->', row.player_RBI_in_match, '-(plays against)->', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location)

Ian Happ <-(from)- Chicago Cubs -(has H in match) 2.0 -(has RBI in match)-> 1.0 -(plays against)-> Colorado Rockies -(plays on)-> 2022-09-16 -(plays at)-> HOME
Randy Arozarena <-(from)- Tampa Bay Rays -(has H in match) 2.0 -(has RBI in match)-> 6.0 -(plays against)-> Toronto Blue Jays -(plays on)-> 2022-09-23 -(plays at)-> HOME
Kyle Farmer <-(from)- Cincinnati Reds -(has H in match) 0.0 -(has RBI in match)-> 0.0 -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-09-25 -(plays at)-> HOME
Jonathan Schoop <-(from)- Detroit Tigers -(has H in match) 2.0 -(has RBI in match)-> 2.0 -(plays against)-> Colorado Rockies -(plays on)-> 2022-04-23 -(plays at)-> HOME
Kyle Farmer <-(from)- Cincinnati Reds -(has H in match) 1.0 -(has RBI in match)-> 2.0 -(plays against)-> Pittsburgh Pirates -(plays on)-> 2022-09-28 -(plays at)-> AWAY
Randy Arozarena <-(from)- Tampa Bay Rays -(has H in match) 0.0 -(has RBI in match)-> 0.0 -(plays against)-> Cleveland Guardians -(plays on)-> 2022-09-29 -(plays at)->

In [26]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# How player (a pitcher) performs in a match
query = """
SELECT DISTINCT ?match ?player1_name ?player_IP_in_match ?player_ER_in_match ?team1_name ?team2_name ?date ?location
WHERE {
    ?match a <http://dsci558.org/class/Match>.

    ?match <http://dsci558.org/ontology/has_player_played> ?player1.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/plays_by> ?team1.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.

    ?player1 <http://dsci558.org/ontology/has_match> ?player_match.
    ?player_match <http://dsci558.org/ontology/has_status> ?player_status_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_status_for_match> ?match.
    ?player_status_in_match <http://dsci558.org/ontology/has_IP> ?player_IP_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_ER> ?player_ER_in_match.
    
    ?team1 <http://dsci558.org/ontology/has_player> ?player1_cluster.
    ?player1_cluster <http://dsci558.org/ontology/for_player> ?player1.

    ?player1 <http://schema.org/name> ?player1_name.
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
LIMIT 10
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.player1_name, '<-(from)-',row.team1_name, '-(has IP in match)', row.player_IP_in_match , '-(has ER in match)->', row.player_ER_in_match, '-(plays against)->', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location)

Shane McClanahan <-(from)- Tampa Bay Rays -(has IP in match) 4.0 -(has ER in match)-> 5.0 -(plays against)-> Houston Astros -(plays on)-> 2022-09-20 -(plays at)-> HOME
Shohei Ohtani <-(from)- Los Angeles Angels -(has IP in match) 7.0 -(has ER in match)-> 1.0 -(plays against)-> Boston Red Sox -(plays on)-> 2022-06-09 -(plays at)-> HOME
Shohei Ohtani <-(from)- Los Angeles Angels -(has IP in match) 3.2 -(has ER in match)-> 6.0 -(plays against)-> Texas Rangers -(plays on)-> 2022-04-14 -(plays at)-> AWAY
Pablo Lopez <-(from)- Miami Marlins -(has IP in match) 7.0 -(has ER in match)-> 0.0 -(plays against)-> St. Louis Cardinals -(plays on)-> 2022-04-21 -(plays at)-> HOME
Shane McClanahan <-(from)- Tampa Bay Rays -(has IP in match) 5.0 -(has ER in match)-> 4.0 -(plays against)-> Toronto Blue Jays -(plays on)-> 2022-09-25 -(plays at)-> HOME
Shohei Ohtani <-(from)- Los Angeles Angels -(has IP in match) 7.0 -(has ER in match)-> 0.0 -(plays against)-> Boston Red Sox -(plays on)-> 2022-05-05 -(plays

In [28]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Prediction - Player Performance - How player (a hitter) performs in a match
query = """
SELECT DISTINCT ?match ?player1_name ?inference_player_RBI_in_match ?player_RBI_in_match ?team1_name ?team2_name ?date ?location
WHERE {
    ?match a <http://dsci558.org/class/Match>.

    ?match <http://dsci558.org/ontology/has_player_played> ?player1.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/plays_by> ?team1.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.

    ?player1 <http://dsci558.org/ontology/has_match> ?player_match.
    ?player_match <http://dsci558.org/ontology/has_status> ?player_status_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_status_for_match> ?match.
    ?player_status_in_match <http://dsci558.org/ontology/inference-has_RBI> ?inference_player_RBI_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_RBI> ?player_RBI_in_match.
    
    ?team1 <http://dsci558.org/ontology/has_player> ?player1_cluster.
    ?player1_cluster <http://dsci558.org/ontology/for_player> ?player1.

    ?player1 <http://schema.org/name> ?player1_name.
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
LIMIT 20
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.player1_name, '<-(from)-',row.team1_name, '-(has RBI in match)', row.player_RBI_in_match , '<-(inference)-', row.inference_player_RBI_in_match, '-(plays against)->', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location)

Ian Happ <-(from)- Chicago Cubs -(has RBI in match) 1.0 <-(inference)- 0.0 -(plays against)-> Colorado Rockies -(plays on)-> 2022-09-16 -(plays at)-> HOME
Randy Arozarena <-(from)- Tampa Bay Rays -(has RBI in match) 6.0 <-(inference)- 0.0 -(plays against)-> Toronto Blue Jays -(plays on)-> 2022-09-23 -(plays at)-> HOME
Kyle Farmer <-(from)- Cincinnati Reds -(has RBI in match) 0.0 <-(inference)- 0.0 -(plays against)-> Milwaukee Brewers -(plays on)-> 2022-09-25 -(plays at)-> HOME
Kyle Farmer <-(from)- Cincinnati Reds -(has RBI in match) 2.0 <-(inference)- 0.0 -(plays against)-> Pittsburgh Pirates -(plays on)-> 2022-09-28 -(plays at)-> AWAY
Randy Arozarena <-(from)- Tampa Bay Rays -(has RBI in match) 0.0 <-(inference)- 0.0 -(plays against)-> Cleveland Guardians -(plays on)-> 2022-09-29 -(plays at)-> AWAY
Kyle Farmer <-(from)- Cincinnati Reds -(has RBI in match) 0.0 <-(inference)- 0.0 -(plays against)-> Chicago Cubs -(plays on)-> 2022-09-30 -(plays at)-> AWAY
Kyle Farmer <-(from)- Cincinnat

In [30]:
import rdflib
filename = "prototype.ttl"

gg = rdflib.Graph()
gg.parse(filename, format='ttl')

# Prediction - Player Performance - How player (a pitcher) performs in a match
query = """
SELECT DISTINCT ?match ?player1_name ?inference_player_ER_in_match ?player_ER_in_match ?team1_name ?team2_name ?date ?location
WHERE {
    ?match a <http://dsci558.org/class/Match>.

    ?match <http://dsci558.org/ontology/has_player_played> ?player1.
    ?match <http://dsci558.org/ontology/plays_at> ?location.
    ?match <http://dsci558.org/ontology/plays_on> ?date.
    ?match <http://dsci558.org/ontology/plays_by> ?team1.
    ?match <http://dsci558.org/ontology/plays_against> ?team2.

    ?player1 <http://dsci558.org/ontology/has_match> ?player_match.
    ?player_match <http://dsci558.org/ontology/has_status> ?player_status_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_status_for_match> ?match.
    ?player_status_in_match <http://dsci558.org/ontology/inference-has_ER> ?inference_player_ER_in_match.
    ?player_status_in_match <http://dsci558.org/ontology/has_ER> ?player_ER_in_match.
    
    ?team1 <http://dsci558.org/ontology/has_player> ?player1_cluster.
    ?player1_cluster <http://dsci558.org/ontology/for_player> ?player1.

    ?player1 <http://schema.org/name> ?player1_name.
    ?team1 <http://schema.org/name> ?team1_name.
    ?team2 <http://schema.org/name> ?team2_name.
}
LIMIT 10
"""

# Can use either text or table to visualize
qres = gg.query(query)
for row in qres:
    print(row.player1_name, '<-(from)-',row.team1_name, '-(has ER in match)', row.player_ER_in_match , '<-(inference)-', row.inference_player_ER_in_match, '-(plays against)->', row.team2_name, '-(plays on)->', row.date, '-(plays at)->', row.location)

Shane McClanahan <-(from)- Tampa Bay Rays -(has ER in match) 5.0 <-(inference)- 1.0 -(plays against)-> Houston Astros -(plays on)-> 2022-09-20 -(plays at)-> HOME
Shane McClanahan <-(from)- Tampa Bay Rays -(has ER in match) 4.0 <-(inference)- 1.0 -(plays against)-> Toronto Blue Jays -(plays on)-> 2022-09-25 -(plays at)-> HOME
Shane McClanahan <-(from)- Tampa Bay Rays -(has ER in match) 1.0 <-(inference)- 2.0 -(plays against)-> New York Yankees -(plays on)-> 2022-05-29 -(plays at)-> HOME
Sandy Alcantara <-(from)- Miami Marlins -(has ER in match) 4.0 <-(inference)- 2.0 -(plays against)-> New York Mets -(plays on)-> 2022-06-24 -(plays at)-> HOME
Corey Kluber <-(from)- Tampa Bay Rays -(has ER in match) 5.0 <-(inference)- 2.0 -(plays against)-> Toronto Blue Jays -(plays on)-> 2022-07-01 -(plays at)-> AWAY
Carlos Rodon <-(from)- San Francisco Giants -(has ER in match) 0.0 <-(inference)- 2.0 -(plays against)-> Chicago Cubs -(plays on)-> 2022-07-31 -(plays at)-> HOME
Pablo Lopez <-(from)- Miami